# ProPyCore SDK Examples: Documents

This notebook contains snippets from the `documents` submodules: `folders.py` and `files.py`.

Procore defines two main document types: folders and files. The `folders` submodule contains functions for working with folders and the `files` submodule contains functions for working with files.

In [1]:
import os
import dotenv
import json

import ProPyCore

## Setup
You will need to create the connection to your Procore app and then get the details for your company and project.

### Connection to Procore App
Ensure you have a `.env` file with the following information included:
* `CLIENT_ID`: your data connection app's client ID
* `CLIENT_SECRET`: your data connection app's client secret

In [2]:
dotenv.load_dotenv()
connection = ProPyCore.procore.Procore(
    client_id=os.getenv("CLIENT_ID"),
    client_secret=os.getenv("CLIENT_SECRET"),
    redirect_uri="urn:ietf:wg:oauth:2.0:oob", # default for data connection apps
    oauth_url="https://app.procore.com", # default for data connection apps
    base_url="https://app.procore.com" # default for data connection apps
)

### Get Company Details
Use the cell below to specify your company name

In [3]:
company_name = "Rogers-O`Brien Construction"

Now get the company details and find a specific project, in this case "Sandbox Test Project".

In [4]:
company = connection.companies.find(identifier=company_name)
project = connection.projects.find(
    company_id=company["id"],
    identifier="Sandbox Test Project"
)
print(f"Project: {project['name']} ({project['id']})")

Project: Sandbox Test Project (2783683)


## Folders

### Get Folders

In [5]:
# Example 1: Get all folders
# ---------
print("Example 1")
folders = connection.folders.get(
    company_id=company["id"],
    project_id=project["id"]
)
print(folders)

Example 1
[{'id': 816142268, 'created_at': '2024-07-17T20:56:14Z', 'created_by': {'id': 6036589, 'company_name': "Rogers-O'Brien Construction Company", 'locale': 'en', 'login': 'sthompson@r-o.com', 'name': 'Scott Thompson'}, 'custom_fields': {}, 'document_type': 'folder', 'is_deleted': False, 'is_recycle_bin': False, 'name': '03-Concrete', 'name_with_path': 'Sandbox Test Project/C-Field Use Documents/03-Concrete', 'parent_id': 771257695, 'private': False, 'read_only': False, 'updated_at': '2024-07-17T20:56:14Z'}, {'id': 845915608, 'created_at': '2024-10-02T15:48:01Z', 'created_by': {'id': 3238394, 'company_name': "Rogers-O'Brien Construction Company", 'locale': 'en', 'login': 'nvargo@r-o.com', 'name': 'Nick Vargo'}, 'custom_fields': {}, 'document_type': 'folder', 'is_deleted': False, 'is_recycle_bin': False, 'name': '1-PreTask Planning', 'name_with_path': 'Sandbox Test Project/G-Safety and Environmental/1-PreTask Planning', 'parent_id': 771257651, 'private': False, 'read_only': False, 

In [6]:
# Example 3: Get all children folders from parent
# ---------
print("\nExample 3")
subfolders = connection.folders.get(
    company_id=company["id"],
    project_id=project["id"],
    folder_id=816142268
)
print(subfolders)



Example 3
[{'id': 816142268, 'created_at': '2024-07-17T20:56:14Z', 'created_by': {'id': 6036589, 'company_name': "Rogers-O'Brien Construction Company", 'locale': 'en', 'login': 'sthompson@r-o.com', 'name': 'Scott Thompson'}, 'custom_fields': {}, 'document_type': 'folder', 'is_deleted': False, 'is_recycle_bin': False, 'name': '03-Concrete', 'name_with_path': 'Sandbox Test Project/C-Field Use Documents/03-Concrete', 'parent_id': 771257695, 'private': False, 'read_only': False, 'updated_at': '2024-07-17T20:56:14Z'}]


### Create Folder

In [7]:
# Example 1: Create folder in Root (no parent_id provided)
# ---------
print("Example 1")
try:
    root_folder = connection.folders.create(
        company_id=company["id"],
        project_id=project["id"],
        folder_name="Z-Research and Development"
    )
    print(f"{root_folder['id']}: {root_folder['name']}")
except Exception as e:
    print(e)
# 607848046: Z-Research and Development

Example 1
'Error: 400'
cannot access local variable 'doc_info' where it is not associated with a value


In [8]:
# Example 2: Create folder in specified location
# ---------
print("\nExample 2")
folder = connection.folders.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier="Z-Research and Development" # this needs to be a path in your procore project
)
# 607848083: A-Team

try:
    subfolder = connection.folders.create(
        company_id=company["id"],
        project_id=project["id"],
        folder_id=folder["id"],
        folder_name="A-Team"
    )
    print(f"{subfolder['id']}: {subfolder['name']}")
except Exception as e:
    print(e)


Example 2
929510338: A-Team


In [9]:
# Example 3: Folder already exists
# ---------
print("\nExample 3")
try:
    existing_folder_name = "I-Safety and Environmental"
    existing_folder = connection.folders.create(
        company_id=company["id"],
        project_id=project["id"],
        folder_name=existing_folder_name
    )
    print(f"{existing_folder['id']}: {existing_folder['name']}")
except WrongParamsError as e:
    print(e)
# 'Folder I-Safety and Environmental already exists'


Example 3
929510340: I-Safety and Environmental


### Find Folder

In [10]:
# Example 1: Find folder in root
# ---------
print("Example 1")
folder1 = connection.folders.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier="Z-Research and Development"
)
print(f"{folder1['id']}: {folder1['name']}")
# 607848046: Z-Research and Development
print(json.dumps(folder1, indent=4))

Example 1
929510260: Z-Research and Development
{
    "id": 929510260,
    "custom_fields": {},
    "name": "Z-Research and Development",
    "name_with_path": "Sandbox Test Project/Z-Research and Development",
    "parent_id": 771257538,
    "updated_at": "2025-05-13T17:48:59Z",
    "is_deleted": false,
    "is_recycle_bin": false,
    "is_tracked": false,
    "tracked_folder": null,
    "has_children": true,
    "has_children_files": false,
    "has_children_folders": true,
    "folders": [
        {
            "id": 929510338,
            "custom_fields": {},
            "name": "A-Team",
            "name_with_path": "Sandbox Test Project/Z-Research and Development/A-Team",
            "parent_id": 929510260,
            "updated_at": "2025-05-13T17:49:08Z",
            "is_deleted": false,
            "is_recycle_bin": false,
            "is_tracked": false,
            "tracked_folder": null,
            "has_children": false,
            "has_children_files": false,
           

In [11]:
# Example 2: Find subfolder
# ----------
print("\nExample 2")
folder2 = connection.folders.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier="Subcontractors Orientation"
)
print(f"{folder2['id']}: {folder2['name']}")
# 607846791: Subcontractors Orientation


Example 2


NotFoundItemError: 'Could not find document Subcontractors Orientation'

In [ ]:
# Example 3: No such folder
# ---------
print("\nExample 3")
try:
    folder3 = connection.folders.find(
        company_id=company["id"],
        project_id=project["id"],
        identifier="Not a folder"
    )
    print(folder3)
except NotFoundItemError as e:  
    print(e)
# 'Could not find document Not a folder'

### Update Folder

In [ ]:
# create a folder
connection.folders.create(
    company_id=company["id"],
    project_id=project["id"],
    folder_name=f"Folder_in_Root"
)

folder = connection.folders.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier="Folder_in_Root"
)

AttributeError: 'Procore' object has no attribute '__folders__'

In [ ]:
# Example 1: Move folder
# ---------
print("Example 1")
subfolder = connection.folders.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier="I-Safety and Environmental"
)

connection.folders.update(
    company_id=company["id"],
    project_id=project["id"],
    doc_id=folder["id"],
    parent_id=subfolder["id"]
)

In [ ]:
# Example 2: Update folder name
# ---------
print("\nExample 2")
connection.folders.update(
    company_id=company["id"],
    project_id=project["id"],
    doc_id=folder["id"],
    folder_name="Now_a_Subfolder"
)

In [ ]:
# Example 3: Change permissions
# ---------
print("\nExample 2")
connection.folders.update(
    company_id=company["id"],
    project_id=project["id"],
    doc_id=folder["id"],
    private=True
)

## Files

### Get Files

In [12]:
# Example 2: Get all files
# ---------
print("\nExample 2")
files = connection.files.get(
    company_id=company["id"],
    project_id=2289314,
    view="extended",
    file_types=["pdf"]
)

print("Number of files:", len(files))


Example 2
Number of files: 7068


In [ ]:
# Example 4: Get all children files from parent
# ---------
print("\nExample 4")
subfiles = connection.files.get(
    company_id=company["id"],
    project_id=project["id"],
    folder_id=607848046
)
print(subfiles)

### Create Files

In [ ]:
# Example 1: Create file in Root (no parent_id provided)
# ---------
print("Example 1")
try:
    file_in_root = connection.files.create(
        company_id=company["id"],
        project_id=project["id"],
        filepath=f"{pathlib.Path(__file__).resolve().parent.parent}/data/test/test_pdf.pdf"
    )
    print(f"{file_in_root['id']}: {file_in_root['name']}")
except WrongParamsError as e:
    print(e)
# 607852186: test_pdf.pdf

In [ ]:
 # Example 2: Create file in specified location
# ---------
print("\nExample 2")
try:
    folder = connection.folders.find(
        company_id=company["id"],
        project_id=project["id"],
        identifier="Subcontractors Orientation" # this needs to be a folder in your procore project
    )

    file = connection.files.create(
        company_id=company["id"],
        project_id=project["id"],
        folder_id=folder["id"],
        filepath=f"{pathlib.Path(__file__).resolve().parent.parent}/data/test/another_test_pdf.pdf"
    )
    print(f"{file['id']}: {file['name']}")
except WrongParamsError as e:
    print(e)
# 607851830: another_test_pdf.pdf

In [ ]:
# Example 3: File already exists
# ---------
try:
    file = connection.files.create(
        company_id=company["id"],
        project_id=project["id"],
        folder_id=folder["id"],
        filepath=f"{pathlib.Path(__file__).resolve().parent.parent}/data/test/another_test_pdf.pdf"
    )
    print(f"{file['id']}: {file['name']}")
except WrongParamsError as e:
    print(e)
# 'File another_test_pdf.pdf already exists'

### Find Files

In [ ]:
# Example 1: Find file in root
# ---------
print("Example 1")
file1 = connection.files.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier="test_pdf.pdf"
)
print(f"{file1['id']}: {file1['name']}")
# 607852186: test_pdf.pdf
print(json.dumps(file1, indent=4))
# See example in /references/

In [ ]:
# Example 2: Find file in subfolder
# ----------
print("\nExample 2")
file2 = connection.files.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier="another_test_pdf.pdf"
)
print(f"{file2['id']}: {file2['name']}")
# 607851830: another_test_pdf.pdf

In [ ]:
# Example 3: No such file
# ---------
print("\nExample 3")
try:
    file3 = connection.files.find(
        company_id=company["id"],
        project_id=project["id"],
        identifier="Not a file.txt"
    )
    print(file3)
except NotFoundItemError as e:
    print(e)
# 'Could not find document Not a file.txt'

In [ ]:
# Example 1: Search for file with multiple perfect matches
# ---------
print("Example 1")
doc1 = connection.files.search(
    company_id=company["id"],
    project_id=project["id"],
    value="test"    
)
print(f"{doc1['id']}: {doc1['name']}")
# warn("Multiple 100% matches - try refining your search critera for better results")
# 607851830: test_pdf.pdf

In [ ]:
# Example 1: Search for private file
# ---------
print("\nExample 2")
doc2 = connection.files.search(
    company_id=company["id"],
    project_id=project["id"],
    value="another"
)
print(f"{doc2['id']}: {doc2['name']}")
# 607851830: another_test_pdf.pdf

In [ ]:
# Example 3: Find folder 
# ---------
print("\nExample 3")
doc3 = connection.folders.search(
    company_id=company["id"],
    project_id=project["id"],
    value="training"
)
print(f"{doc3['id']}: {doc3['name']}")
# 607846718: 3-Orientations and Training

In [ ]:
# Example 5: Find file in subfolder 
# ---------
print("\nExample 5")
folder = connection.folders.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier="I-Safety and Environmental"
)
doc4 = connection.files.search(
    company_id=company["id"],
    project_id=project["id"],
    folder_id=folder["id"],
    value="test"
)
print(f"{doc4['id']}: {doc4['name']}")
# 607851830: another_test_pdf.pdf

In [ ]:
# Example 5: Find subfolder in specified folder 
# ---------
print("\nExample 5")
folder = connection.folders.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier="I-Safety and Environmental"
)
doc5 = connection.folders.search(
    company_id=company["id"],
    project_id=project["id"],
    folder_id=folder["id"],
    value="subcontractor"
)
print(f"{doc5['id']}: {doc5['name']}")
# 607846791: Subcontractors Orientation

### Update Files

In [ ]:
# Housekeeping
# ------------
# start by deleting the test files if they are present
for old_file in ["test_pdf.pdf", "renamed_test_pdf", "another_test_pdf.pdf"]:
    # look for the filename
    try:
        file_temp = connection.files.find(
            company_id=company["id"],
            project_id=project["id"],
            identifier=old_file
        )
        
        response = connection.files.remove(
            company_id=company["id"],
            project_id=project["id"],
            doc_id=file_temp["id"],
        )
        print(f"Delete {old_file}:", response["status_code"])
    except NotFoundItemError as e:
        print(e)

# create a file
try:
    connection.files.create(
        company_id=company["id"],
        project_id=project["id"],
        filepath=f"{pathlib.Path(__file__).resolve().parent.parent}/data/test/test_pdf.pdf",
        description="Nothing to see here"
    )
except WrongParamsError as e:
    print(e)

file_original = connection.files.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier="test_pdf.pdf"
)

In [ ]:
# Example 1: Move file
# ---------
print("Example 1")
subfolder = connection.folders.find(
    company_id=company["id"],
    project_id=project["id"],
    identifier="I-Safety and Environmental"
)

file_new_loc = connection.files.update(
    company_id=company["id"],
    project_id=project["id"],
    doc_id=file_original["id"],
    folder_id=subfolder["id"]
)
print(f"Original parent ID:", file_original["parent_id"])
print(f"Updated parent ID:", file_new_loc["parent_id"])

In [ ]:
# Example 2: Update file name
# ---------
print("\nExample 2")
file_new_name = connection.files.update(
    company_id=company["id"],
    project_id=project["id"],
    doc_id=file_original["id"],
    filename="renamed_test_pdf.pdf"
)
print(f"Original filename:", file_original["name"])
print(f"Updated filename:", file_new_name["name"])

In [ ]:
# Example 3: Change Description
# ---------
print("\nExample 3")
file_new_desc = connection.files.update(
    company_id=company["id"],
    project_id=project["id"],
    doc_id=file_original["id"],
    description="This document now has a fancy description"
)
print(f"Original description:", file_original["description"])
print(f"Updated description:", file_new_desc["description"])

In [ ]:
# Example 4: Change permissions
# ---------
print("\nExample 4")
file_new_permissions = connection.files.update(
    company_id=company["id"],
    project_id=project["id"],
    doc_id=file_original["id"],
    private=False
)
print(f"Private? (Original):", file_original["private"])
print(f"Private? (Updated):", file_new_permissions["private"])

In [ ]:
# Example 5: Update file content
# ---------
print("\nExample 5")
file_new_content = connection.files.update(
    company_id=company["id"],
    project_id=project["id"],
    doc_id=file_original["id"],
    filepath=f"{pathlib.Path(__file__).resolve().parent.parent}/data/test/another_test_pdf.pdf"
)
print("Original Number of Versions:", len(file_original["file_versions"]))
print("Updated Number of Versions:", len(file_new_content["file_versions"]))

---